<img src="VSM_logo.gif" width="500" align="center">

# VSM - Volcanic and Seismic source Modelling
### Version 1.0
VSM is a Python tool to perform inversions of geodetic data.

This Notebook runs VSM from the input file already created. It also contains code to compute volume variations, dislocation vertices and additional plots. No need to run VSM if you want just to compute the volume variation of a source or create plots. Just make sure to run the first cell at the beginning. \
Use https://github.com/EliTras/VSM/blob/main/VSM/VSM_writeinput.ipynb for step-by-step instructions to create an input file, or launch the VSM GUI.

**Code** https://github.com/EliTras/VSM \
**License** https://github.com/EliTras/VSM/blob/main/license.lic

----
Run the next cell in any case!

In [ ]:
import os
#import sys
#sys.path.append('path-to-VSM-folder')
import VSM
import VSM_utilities as VSMU

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

## Launch VSM

No need to run the following cells if you want to do post-processing. Just go to **Utilities**

#### Insert input file

In [ ]:
folder_work = '../USECASE/mogi1NA'

filename_in = 'VSM_input_mogi.txt'

#### Read all inputs to populate all variables of VSM

In [ ]:
VSM.read_VSM_settings(os.path.join(folder_work, filename_in))

#### Launch the execution of VSM

In [ ]:
VSM.iVSM()

----------
## Utilities

Here are some utilities to compute the volume variation of the sources from their shape and overpressure vs shear modulus ratio ($\Delta P/ \mu$). Also, it computes the vertices of the fault/dike.

In particular
- Volume variation of the finite volume sphere
- Volume variation of the finite volume spheroid
- Volume variation of the sill in point-source approximation
- Rectangular dislocation vertices and trace

Hint: just run the box you need!

**Bibliography**

Amoruso, A., Crescentini, L. (2013). Analytical models of volcanic ellipsoidal expansion sources. Ann. Geophys., https://doi.org/10.4401/ag-6441

Battaglia, M., Cervelli, P.F., Murray, J.R. (2013). Modeling Crustal Deformation near Active Faults and Volcanic Centers—A  Catalog of Deformation Models, Book 13, Volcanic Monitoring, USGS. 

Okada, Y. (1985). Surface deformation due to shear and tensile faults in a half-space. B. Seismol. Soc. Am. 75, 1135-1154.

#### Volume variation of the spheroid
Define semi-major axis, $\Delta P/ \mu$ and ratio between axes (ratio < 1)

In [ ]:
a = 1300
dP_mu = 7.5e-3
ratio = 0.3

dVol = VSMU.volume_var_yang(dP_mu,a,ratio)
print('Volume variation of the finite volume spheroid',dVol/1e6, '10e6 m3')

#### Volume variation of the sphere
Define radius, $\Delta P/ \mu$ and depth

In [ ]:
radius = 800
dP_mu = 3.1e-3
depth = 2550
    
dVol = VSMU.volume_var_mctigue(dP_mu,radius,depth)
print('Volume variation of the finite volume sphere',dVol/1e6, '10e6 m3')

#### Volume variation of the penny-shaped crack
Define radius, $\Delta P/ \mu$ and Poisson Coefficient

In [ ]:
# holds for depth/radius > 2 or 2.5all
radius = 820
dP_mu = 5.94e-3
Poisson = 0.25

dVol = VSMU.volume_var_penny(dP_mu,radius,Poisson)
print('Volume variation of the penny-shaped crack',dVol/1e6,' 10e6 m3')

#### Vertices of the dislocation from the top left corner
Define Top Left Corner (TLC) coordinates, length, width, strike and dip of the plane. Obtain Bottom Left Corner (BLT), Bottom Right Corner (BRC), Top Right Corner (TRC), and the Trace Left & Right coordinates

In [ ]:
TLC = [7.48873e+05,9.82e+06,2.21526e+03]
length = 9000.
width  = 4000.
strike = 180.
dip    = 83.
    
vert,trace = VSMU.fault_vertices(TLC,length,width,strike,dip)
print('Fault vertices ---------------------')
print('Fault TLC',vert[0])
print('Fault BLC',vert[1])
print('Fault BRC',vert[2])
print('Fault TRC',vert[3])
print('Trace ------------------------------')
print('Trace Left', trace[0])
print('Trace Right',trace[1])

----
## Plots

Here are two code blocks to plot
- 1D and 2D statistics of the parameters searched using the corner library. May be used to change the number of burn-in models
- Quick plot of the comparisons among oberved data, synthetic data and residuals.

#### Plot 1D and 2D statistics
It creates the same plot as VSM. May be used to change the number of burn-in models.

Should set
- working folder
- file with the ensample of models generated
- file of mean or best model (based on the inversion chosen in VSM)
- burn-in models

In [ ]:
# PLOT 1D - 2D statistics via corner
folder = '../USECASE/mogi1NA'
f_models ='VSM_models.csv'
# best (for NA) or mean (for BI)
f_truths = 'VSM_best.csv'
f_plot = 'VSM_plot1D2D_new.png'  #can use pdf or png extension
nskip = 1000

df = pd.read_csv(os.path.join(folder,f_models))
tf = pd.read_csv(os.path.join(folder,f_truths))
nunk = np.size(tf)
    
if(f_truths[4:8] == 'best'):
    dd = df.iloc[:,2:] # NA
    tt = tf.values.reshape(nunk) #NA
else:
    dd = df #BI
    tp = tf.values.reshape(nunk)
    tt = tp[::2] #BI
    
VSM.plot_1D_2D(dd, tt, dd.columns, os.path.join(folder,f_plot), nskip)

#### Plot Data, model and residuals

The following code creates 3 plots automatically scaled to data extension. The figure is composed of three panels: observed data, modelled data (from optimal model) and residuals, e.g., data minus modelled data.
Should set
- working folder
- file with synthetic data. It works for SAR and levelling data as it takes the third column of the synth file as modelled data. Should be rearranged for other geodetic data.
- UTM zone

In [ ]:
folder_inout = '../USECASE/mogi1NA'
synth_file = 'VSM_synth_sar2.csv'
out_file = 'VSM_res_sar2.png'

# UTM zone
zone = 33
southern_hemisphere = False

In [ ]:
# My data in UTM coordinates
db_sar = pd.read_csv(os.path.join(folder_inout,synth_file))
d_sar = db_sar.values

# Split into east and north coordinates
east, north = d_sar[:,0],d_sar[:,1]
data = d_sar[:,3]
synth = d_sar[:,2]
res = data - synth

dmax = max(max(data),max(synth))
dmin = min(min(data),min(synth))
resmax = max(res)
resmin = min(res)
resext = max(resmax, -resmin)

# Define the projection
#crs=ccrs.PlateCarree()
mycrs=ccrs.UTM(zone=zone, southern_hemisphere=southern_hemisphere)

fig=plt.figure(figsize=(15,6))
palette='viridis' #'RdYlBu_r'

## PANEL DATA ##########
ax = plt.subplot(131, projection=mycrs)
ax.coastlines(resolution='10m')
img = ax.scatter(east, north,5, data,cmap=palette, vmin=dmin, vmax = dmax)
#palette
cbar = plt.colorbar(img,orientation='horizontal')
cbar.set_label('LOS (m)')
# Get the extent of the axis
extent = ax.get_extent()
# Attempt to set the axis extent
ax.set_extent(extent, crs=mycrs)
plt.title('Data',fontsize = 16, pad=10)

## PANEL MODEL ##########
ax = plt.subplot(132, projection=mycrs)
ax.coastlines(resolution='10m')
img = ax.scatter(east, north,5, synth,cmap=palette, vmin=dmin, vmax = dmax)
#palette
cbar = plt.colorbar(img,orientation='horizontal')
cbar.set_label('LOS (m)')
# Get the extent of the axis
extent = ax.get_extent()
# Attempt to set the axis extent
ax.set_extent(extent, crs=mycrs)
plt.title('Model',fontsize = 16, pad=10)

## PANEL RESIDUALS ##########
ax = plt.subplot(133, projection=mycrs)
ax.coastlines(resolution='10m')
img = ax.scatter(east, north,5, res,cmap="bwr",vmin=resext, vmax = -resext)
#palette
cbar = plt.colorbar(img,orientation='horizontal')
cbar.set_label('LOS (m)')
# Get the extent of the axis
extent = ax.get_extent()
# Attempt to set the axis extent
ax.set_extent(extent, crs=mycrs)
# Title for plot
plt.title('Residual',fontsize = 16, pad=10)

plt.savefig(os.path.join(folder_inout,out_file))
plt.show()